In [83]:
%pip install pandas

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/opt/homebrew/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [84]:
import pandas as pd

In [85]:
path = "../data/combined/"


In [86]:
train_df = pd.read_json(path + "train.json")
test_df = pd.read_json(path + "test.json")


In [87]:
train_df

,text,entities
0,i want to fly from boston at 838 am and arrive...,"[{'start': 19, 'end': 25, 'value': 'boston', '..."
1,what flights are available from pittsburgh to ...,"[{'start': 2, 'end': 2, 'value': 'thursday', '..."
2,what is the arrival time in san francisco for ...,"[{'start': 12, 'end': 24, 'value': 'arrival ti..."
3,cheapest airfare from tacoma to orlando,"[{'start': 1, 'end': 9, 'value': 'cheapest', '..."
4,round trip fares from pittsburgh to philadelph...,"[{'start': 1, 'end': 11, 'value': 'round trip'..."
...,...,...
10195,Biersack,"[{'entity': 'last_name', 'start': 0, 'end': 7,..."
10196,Four persons will be dining in our party.,"[{'entity': 'people', 'start': 0, 'end': 3, 'v..."
10197,in 12 more days,"[{'entity': 'date', 'start': 0, 'end': 14, 'va..."
10198,18,"[{'entity': 'time', 'start': 0, 'end': 1, 'val..."


In [88]:
test_df

,text,entities
0,i would like to find a flight from charlotte t...,"[{'start': 35, 'end': 44, 'value': 'charlotte'..."
1,on april first i need a ticket from tacoma to ...,"[{'start': 0, 'end': 1, 'value': 'april first'..."
2,on april first i need a flight going from phoe...,"[{'start': 0, 'end': 1, 'value': 'april first'..."
3,i would like a flight traveling one way from p...,"[{'start': 3, 'end': 4, 'value': 'april first'..."
4,i would like a flight from orlando to salt lak...,"[{'start': 2, 'end': 3, 'value': 'april first'..."
...,...,...
3264,The Cazaree party of two,"[{'entity': 'last_name', 'start': 4, 'end': 10..."
3265,Use the name Anne Pegg,"[{'entity': 'first_name', 'start': 13, 'end': ..."
3266,Book under Elanor Dinkel,"[{'entity': 'first_name', 'start': 11, 'end': ..."
3267,Karla Trevizo i don't remember the phone number,"[{'entity': 'first_name', 'start': 0, 'end': 4..."


In [89]:
tokens = []
ner_tags = []

for i in range(len(train_df)):
    token = train_df["text"][i].replace(";", " ").split()
    
    # remove dot in last word in reverse order
    if token[-1][-1] == ".":
        token[-1] = token[-1][0:len(token[-1])-1]
    ner_tag = ['0' for i in range(len(token))]
    for j in range(len(train_df["entities"][i])):
        values = train_df["entities"][i][j]["value"].split()
        for value in values:
            if value in token:
                ner_tag[
                    token.index(
                        value
                    )
                ] = train_df["entities"][i][j]["entity"]
    tokens.append(token)
    ner_tags.append(ner_tag)


In [90]:
onto_bert_train_df = pd.DataFrame({
    "tokens": tokens,
    "ner_tags": ner_tags,
}
)
onto_bert_train_df


,tokens,ner_tags
0,"[i, want, to, fly, from, boston, at, 838, am, ...","[0, 0, 0, 0, 0, fromloc.city_name, 0, time, ti..."
1,"[what, flights, are, available, from, pittsbur...","[0, 0, 0, 0, 0, fromloc.city_name, 0, toloc.ci..."
2,"[what, is, the, arrival, time, in, san, franci...","[0, 0, 0, flight_time, flight_time, 0, fromloc..."
3,"[cheapest, airfare, from, tacoma, to, orlando]","[cost_relative, 0, 0, fromloc.city_name, 0, to..."
4,"[round, trip, fares, from, pittsburgh, to, phi...","[round_trip, round_trip, 0, 0, fromloc.city_na..."
...,...,...
10195,[Biersack],[last_name]
10196,"[Four, persons, will, be, dining, in, our, party]","[people, 0, 0, 0, 0, 0, 0, 0]"
10197,"[in, 12, more, days]","[date, date, date, date]"
10198,[18],[time]


In [91]:
tokens_test = []
ner_tags_test = []

for i in range(len(test_df)):
    token = test_df["text"][i].replace(";", " ").split()

    # remove dot in last word in reverse order
    if token[-1][-1] == ".":
        token[-1] = token[-1][0:len(token[-1])-1]
    ner_tag = ['0' for i in range(len(token))]
    for j in range(len(test_df["entities"][i])):
        values = test_df["entities"][i][j]["value"].split()
        for value in values:
            if value in token:
                ner_tag[
                    token.index(
                        value
                    )
                ] = test_df["entities"][i][j]["entity"]
    tokens_test.append(token)
    ner_tags_test.append(ner_tag)


In [92]:
onto_bert_test_df = pd.DataFrame({
    "tokens": tokens_test,
    "ner_tags": ner_tags_test,
}
)
onto_bert_test_df


,tokens,ner_tags
0,"[i, would, like, to, find, a, flight, from, ch...","[0, 0, 0, 0, 0, 0, 0, 0, fromloc.city_name, 0,..."
1,"[on, april, first, i, need, a, ticket, from, t...","[0, date, date, 0, 0, 0, 0, 0, fromloc.city_na..."
2,"[on, april, first, i, need, a, flight, going, ...","[0, date, date, 0, 0, 0, 0, 0, 0, fromloc.city..."
3,"[i, would, like, a, flight, traveling, one, wa...","[0, 0, 0, 0, 0, 0, round_trip, round_trip, 0, ..."
4,"[i, would, like, a, flight, from, orlando, to,...","[0, 0, 0, 0, 0, 0, fromloc.city_name, 0, toloc..."
...,...,...
3264,"[The, Cazaree, party, of, two]","[0, last_name, 0, 0, people]"
3265,"[Use, the, name, Anne, Pegg]","[0, 0, 0, first_name, last_name]"
3266,"[Book, under, Elanor, Dinkel]","[0, 0, first_name, last_name]"
3267,"[Karla, Trevizo, i, don't, remember, the, phon...","[first_name, last_name, 0, 0, 0, 0, 0, 0]"


In [93]:
onto_bert_train_df.to_json(path + "onto_bert_train.json")
onto_bert_test_df.to_json(path + "onto_bert_test.json")
